In [1]:
!pip install pandas seaborn scikit-image opencv-python scikit-learn tensorflow

In [14]:
import os

import numpy as np
import pandas as pd
# Torch
import torch
import torchvision
from torchvision import datasets,transforms
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import pathlib
from torchvision import models
from sklearn.metrics import precision_score, f1_score

In [3]:
transformer = transforms.Compose([transforms.Resize((150,150)),
                                  transforms.Grayscale(num_output_channels=1),
                                  transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(), # chnages from 0-255 to 0-1, numpy to tensors
                                  transforms.Normalize([0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5])
                                 ])

In [4]:
# Check if CUDA is available and set the device accordingly
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)  # Print the device being used

cuda:0


In [5]:
batchSize=64
test_path = 'data/test'
test_loader = DataLoader(torchvision.datasets.ImageFolder(
                          test_path,
                          transform=transformer),
                          batch_size = batchSize,
                          shuffle=False)

In [6]:
total_images_test = len(test_loader.dataset)

print("Total number of images in the test dataset:", total_images_test)

Total number of images in the test dataset: 624


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomCNN(nn.Module):
    def __init__(self, img_channels, class_count):
        super(CustomCNN, self).__init__()
        # 150×150
        
        # Convolutional layers
        self.conv_layers = nn.Sequential(
            nn.Conv2d(img_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), #75×75
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), #37×37
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), #18×18
            
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), #9×9
            
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), #4×4
        )
        
        # Fully connected layers
        self.fc_layers = nn.Sequential(
            nn.Linear(512 * 4 * 4, 256),  # Adjust the linear layer input size according to your image input size
            nn.ReLU(inplace=True),
            nn.Linear(256, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, class_count)
        )
        
    def forward(self, x):
        # Pass input through conv layers
        x = self.conv_layers(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Pass input through fc layers
        x = self.fc_layers(x)
        
        return x


In [17]:
model = CustomCNN(1, 2).to(device)

In [18]:
#loading saved model

model_path = 'paths/model_adagrad_0001_128_relu.pth'
state_dict = torch.load(model_path,map_location=device)

model.load_state_dict(state_dict)
model.to(device)

#to calulate eval later
predicted_list = []
target_list =[]
#set to evaluation mode
model.eval()

total_correct = 0
total_images_test = len(test_loader.dataset)
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == targets).sum().item()
        predicted_list.extend(predicted.cpu().numpy())
        target_list.extend(targets.cpu().numpy())

accuracy = total_correct / total_images_test
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 87.66%


In [19]:
precision = precision_score(target_list, predicted_list, average='macro')
print(f'Precision: {precision * 100:.2f}%')

f1 = f1_score(target_list, predicted_list, average='macro')
print(f'F1 Score: {f1 * 100:.2f}%')

Precision: 86.82%
F1 Score: 86.85%
